# pixabay.com image 크롤링
- https://pixabay.com/
    - 무료 이미지 제공 사이트
    - 메인 페이지의 맨 아래 버튼을 클릭하면 전체 이미지를 볼 수 있다.
        - url: https://pixabay.com/images/search/?pagi=페이지번호
    - 전체 페이지 수는 **상단 오른쪽**에 보인다.
    - 특정 주제의 이미지만 검색후 볼 수 있다.
        - url: https://pixabay.com/images/search/검색어/
- img 의 링크는 src가 /static/img/blank.gif 인경우 data-lazy 속성을 읽는다. (이미지가 지연로딩처리되 있음)
- 이미지 지연로딩
    - 현재 보이는 화면에 나오는 이미지만 로딩하고 화면밖의 이미지는 화면에 나올때 나중에 로딩시켜 초기 로딩속도를 개선하는 방식
    

1. 메인페이지 이미지 크롤링
2. 전체 페이지중 N 페이지까지의 이미지 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import os

In [ ]:


response = requests.get(url, headers=headers)

In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
baseurl = 'https://pixabay.com'
res = requests.get(baseurl, headers=headers)
img_directory = './pixabay_img' #저장 디렉토리
#       저장 디렉토리 확인후 없으면 만들기
if not os.path.isdir(img_directory):
    os.mkdir(img_directory)

- 이미지 컨테이너: div.flex_grid.credits
- 각 이미지의 컨테이너 : 이미지 컨테이너 내의 div.item
- 이미지: 각 이미지 컨테이너(div.item) 안의 `<a> 내의 <img>`
    - img의 src 속성에 이미지 링크가 있다. 
    - src='/static/img/blank.gif' 인 것들은 lazy 로딩이다. 나중에 로딩할 이미지 경로를 data-lazy 속성으로 제공

- **tag.content**: binary 
- **tag.text**: text

In [10]:
res.status_code

403

In [8]:
print(res.status_code)
if res.status_code==200:    
    soup = BeautifulSoup(res.text, 'html.parser')
#     print(soup.prettify());
    image_elem_list = soup.select('.photo-result-image')
    
    for img in image_elem_list:
#       

#       img태그의 src 속성 조회 (이미지 경로 조회 - lazy loading 을 이용했기 때문에 조회결과가 /static/img/blank.gif 인경우 data-lazy 속성을 조회)
        img_src = img.get('src').strip()
        print(img_src)
#         print(img_src) #출력해서 /static/img/blank.gif (자리잡기위한 이미지) 것들 보여주고 이미지 지연로딩 설명

        if img_src == '/static/img/blank.gif':
            img_src = img.get('data-lazy').strip()
        
#       이미지 경로에서 파일명만 추출(저장시 사용)
        tmp = img_src.split('/')
        file_name = tmp[len(tmp)-1]
          
#      이미지 요청해서 파일로 저장
        with open(os.path.join(img_directory, file_name), 'wb') as f:
            img_res = requests.get(img_src)
            if img_res.status_code == 200:
                f.write(img_res.content) #binary는 content로 읽는다.

403


In [6]:
!cd pixabay_img
!dir pixabay_img

 E 드라이브의 볼륨: samsung_ssd
 볼륨 일련 번호: 5C1D-BF98

 E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\02_수업소스\pixabay_img 디렉터리

2023-03-17  오전 11:23    <DIR>          .
2023-03-17  오전 11:23    <DIR>          ..
               0개 파일                   0 바이트
               2개 디렉터리  58,679,697,408 바이트 남음


# TODO pixabay 이미지 다운로드 2
- 메인페이지 Discover more 클릭하면 제공 이미지 모두를 받을 수 있다. 
- 페이지는 페이징 처리되어 Next 버튼을 클릭하면 다음페이지로 이동한다.
- 10 page의 image들을 다운 받는다.

In [63]:
def download_pixabay_images(html_doc):
    '''
    pixabay 의 image 페이지를 받아서 이미지를 다운 받아 pixabay_img 폴더에 저장한다.
    '''
    soup = BeautifulSoup(html_doc, 'lxml')
#     print(soup.prettify());
    div_items = soup.select('div.flex_grid.credits > div.item')
    
    for div_item in div_items:
#       div 하위의 img 태그 조회
        img = div_item.find('img')

#       img태그의 src 속성 조회 (이미지 경로 조회 - lazy loading 을 이용했기 때문에 조회결과가 /static/img/blank.gif 인경우 data-lazy 속성을 조회)
        img_src = img.get('src').strip()
        if img_src == '/static/img/blank.gif':
            img_src = img.get('data-lazy').strip()
        
#       이미지 경로에서 파일명만 추출(저장시 사용)
        tmp = img_src.split('/')
        file_name = tmp[len(tmp)-1]
        
#       저장 디렉토리 확인후 없으면 만들기
        img_directory = './pixabay_img'
        if not os.path.isdir(img_directory):
            os.mkdir(img_directory)
        
#       이미지 요청해서 파일로 저장
        with open(os.path.join(img_directory, file_name), 'wb') as f:
            img_res = requests.get(img_src)
            if img_res.status_code == 200:
                f.write(img_res.content)

In [64]:
url = 'https://pixabay.com/images/search/?pagi={}'
for page in range(30):
    url = url.format(page)
    res = requests.get(url)
    if res.status_code == 200:
        download_pixabay_images(res.text)

PermissionError: [Errno 13] Permission denied: './pixabay_img\\heart-4366897__340.jpg'